In [2]:
import pandas as pd
import unicodedata
import numpy as np
from geopy.geocoders import Nominatim

df = pd.read_csv('./input/main_lodz_full20230802-143112.csv',delimiter=';')


In [118]:
def type_clasificator(who:str):
    who = who.lower()
    if 'oferta prywatna' in who:
        return 'P'
    if 'inwestycja deweloperska' in who:
        return 'D'
    if 'biuro nieruchomości' in who:
        return 'N'
    return 'X'



In [119]:
df.head()


,date,link,name,where,price,perm,rooms,sqm,who
0,20230802,/pl/oferta/mieszkanie-54-m-lodz-ID4mkSG,"Mieszkanie, 54 m², Łódź","Mileszki, Widzew, Łódź, łódzkie",420 000 zł,7778 zł/m²,2 pokoje,54 m²,Oferta prywatna
1,20230802,/pl/oferta/3-pokojowe-mieszkanie-72m2-balkon-b...,3-pokojowe mieszkanie 72m2 + balkon Bezpośrednio,"ul. Mania, Zdrowie, Polesie, Łódź, łódzkie",Zapytaj o cenę,NaN,3 pokoje,72.39 m²,Esembla GroupInwestycja deweloperska
2,20230802,/pl/oferta/2-pokojowe-mieszkanie-38m2-balkon-I...,2-pokojowe mieszkanie 38m2 + balkon,"ul. Mania, Zdrowie, Polesie, Łódź, łódzkie",Zapytaj o cenę,NaN,2 pokoje,38.99 m²,Esembla GroupInwestycja deweloperska
3,20230802,/pl/oferta/3-pokojowe-mieszkanie-46m2-balkon-I...,3-pokojowe mieszkanie 46m2 + balkon,"ul. Hipoteczna, Stare Bałuty, Bałuty, Łódź, łó...",Zapytaj o cenę,NaN,3 pokoje,46.42 m²,Maxbud NoveoInwestycja deweloperska
4,20230802,/pl/oferta/4-pokojowe-mieszkanie-75m2-3-balkon...,4-pokojowe mieszkanie 75m2 + 3 balkony,"ul. Hipoteczna, Stare Bałuty, Bałuty, Łódź, łó...",Zapytaj o cenę,NaN,4 pokoje,75.45 m²,Maxbud NoveoInwestycja deweloperska


In [120]:
# unicode normalization
df['perm'] = df['perm'].fillna('brak').apply(lambda x: unicodedata.normalize('NFKC',x)).replace('brak',np.nan)
df['price'] = df['price'].fillna('brak').apply(lambda x: unicodedata.normalize('NFKC',x)).replace('brak',np.nan).str.replace(',','.')


In [121]:
# enrich fields
df['link'] = 'http://www.otodom.pl' + df['link']
df['sqm'] = df['sqm'].str.replace(' m²','').astype(float)
df['perm'] = df['perm'].str.replace('zł/m2','').str.replace(' ','').fillna(-1).astype(int)
df['price'] = df['price'].str.replace('zł','').str.replace('Zapytaj o cenę','-1').str.replace(' ','').fillna(-1).astype(float).astype(int)
df['rooms'] = df['rooms'].str.replace(r' pokoje| pokoi| pokój','',regex=True).str.replace('+','').astype(int)
# add new
df['id'] = df['link'].str[-7:]
df['type'] = df['who'].apply(type_clasificator)
# type adjustment
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# clean
df.replace({'who' : { 'Inwestycja deweloperska' : '', 'Biuro nieruchomości' : '' }}, inplace=True,regex=True)

C:\Users\krawc\AppData\Local\Temp\ipykernel_224\2400223606.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rooms'] = df['rooms'].str.replace(r' pokoje| pokoi| pokój','',regex=True).str.replace('+','').astype(int)


In [3]:
#Geolocation
geolocator = Nominatim(user_agent="http")

def geolocate(adress:str):
    try:
        adress = adress.lower().replace('ul.','')
        nominatim = geolocator.geocode(adress,country_codes='pl',exactly_one=True)
        return (nominatim.latitude,nominatim.longitude)
    except:
        pass

# df['geo'] = df['where'].apply(geolocate)

In [43]:
ex= 'Mania, Zdrowie, Polesie, Łódź, łódzkie'
rr= 'ul. Hipoteczna, Stare Bałuty, Bałuty, Łódź, łódzkie'
oto=rr.split(',')
x= geolocator.geocode(ex,country_codes='pl',exactly_one=True)
x
def after_dot(string:str):
    return string[string.find('.')+1:]

query_dict = {'street':after_dot(oto[0]),
'city':oto[-2],
'county':oto[-3],
'state':oto[-1],
'country':'pl'}
y= geolocator.geocode(query_dict,country_codes='pl',exactly_one=True)

In [11]:
x.raw['display_name']

'Mania, Zdrowie-Mania, Łódź-Polesie, Łódź, województwo łódzkie, 94-248, Polska'

In [46]:

location_string = y.raw['display_name'].split(', ')
names=['ulica','osiedle','rejon','dzielnica','miasto','wojewodztwo','kod','kraj']


'Hipoteczna, Berlinek, Bałuty-Centrum, Łódź-Bałuty, Łódź, województwo łódzkie, 91-338, Polska'